<font size=5>Run Text_Matcher Algorithm</font>

In [14]:
#Troubleshooting: !jupyter notebook --NotebookApp.iopub_data_rate_limit=1.0e10 if you 
# get a message about the data rate limit
from matcher import Text, Matcher
import json
import pandas as pd
import numpy as np
from IPython.display import clear_output

In [4]:
# ACTION: copy path to corpus data JSONL file here (filename should end "_fulltext.jsonl")

startData = "/Users/milan/Library/CloudStorage/GoogleDrive-mtt2126@columbia.edu/My Drive/iAnnotate/MIT/Quotable Content/Data/Barthes/1968_RealityEffect/Corpus/Barthes_1968_RealityEffect_fulltext.jsonl"

In [5]:
# Infer naming variables from path

textTitle = startData.rsplit("_", 3)[-2]
publicationYear = startData.rsplit("_", 3)[-3]
authorSurname = startData.rsplit("_", 3)[-4]
authorSurname = authorSurname.rsplit("/", 1)[-1]
dataDir = startData.rsplit("/", 4)[0]

print(f"Author surname: {authorSurname}\nPublication year: {publicationYear}\nText title: {textTitle}\nData directory:{dataDir}")

projectName = f"{authorSurname}_{publicationYear}_{textTitle}"
sourceDir = f"{dataDir}/{authorSurname}/{publicationYear}_{textTitle}/Source"
corpusDir = f"{dataDir}/{authorSurname}/{publicationYear}_{textTitle}/Corpus"
resultsDir = f"{dataDir}/{authorSurname}/{publicationYear}_{textTitle}/Results"

Author surname: Barthes
Publication year: 1968
Text title: RealityEffect
Data directory:/Users/milan/Library/CloudStorage/GoogleDrive-mtt2126@columbia.edu/My Drive/iAnnotate/MIT/Quotable Content/Data


In [6]:
# Specify hyperparameters for matcher algorithm (adjust if desired)

thresh = 2
cut = 3
ngram = 2
mindist = 3
nostops = True

hyperparSuffix = f"_t{thresh}-c{cut}-n{ngram}-m{mindist}-{'nostops' if nostops else 'stops'}"
print(f"Suffix to be appended to results: {hyperparSuffix}")

Suffix to be appended to results: _t2-c3-n2-m3-nostops


In [5]:
# Load the text you want to find quotations from.

sourceText = f"{sourceDir}/{projectName}_plaintext.txt"
with open(sourceText) as f: 
    rawText = f.read()

tx = Text(rawText, projectName)

# Load the corpus you want to find results in

corpusFile = f"{corpusDir}/{projectName}_fulltext.jsonl"
with open(corpusFile) as f:
    rawProcessedData = f.readlines()
data = [json.loads(line) for line in rawProcessedData]

In [9]:
# Save as JSONL file for analysis and visualization

df.to_json(path_or_buf=f"{resultsDir}/{projectName}_results{hyperparSuffix}.jsonl", orient='records', lines=True)

# Drop phrases

In [295]:
# Read full text and results

sourceText = "/Users/milan/Library/CloudStorage/GoogleDrive-mtt2126@columbia.edu/My Drive/iAnnotate/MIT/Quotable Content/Data/Genette/1980_NarrativeDiscourse/Source/Genette_1980_NarrativeDiscourse_plaintext.txt"
with open(sourceText) as f: 
    rawText = f.read()


df = pd.read_json("/Users/milan/Library/CloudStorage/GoogleDrive-mtt2126@columbia.edu/My Drive/iAnnotate/MIT/Quotable Content/Data/Genette/1980_NarrativeDiscourse/Results/Genette_1980_NarrativeDiscourse_results_t2-c3-n2-m3-nostops.jsonl", lines=True)

In [296]:
# Tally matches

# Calculate length of source text

textALength = len(rawText)

# Make an empty array the size of the text

tally = np.zeros(textALength)
#tally = [0] * textALength

# Read the matched locations from the results dataset, and literally evaluate them into lists. 

locationsInA = df['Locations in A']

# Tally up every time a letter in the text is quoted. 
for article in locationsInA: 
    for locRange in article: 
         for i in range(locRange[0], min(locRange[1]+1, len(tally))):
                tally[i] += 1

In [304]:
# Specify the range of frequencies to examine (e.g. 0 to 10 for top 10 most frequent)

freqUpper = 0
freqLower = 50

In [305]:
# Identify highest frequencies in descending order
topFreqs = sorted(set(tally), reverse=True)[freqUpper:freqLower]

print(topFreqs)

[736.0, 735.0, 730.0, 712.0, 663.0, 660.0, 659.0, 555.0, 554.0, 546.0, 536.0, 412.0, 398.0, 383.0, 381.0, 380.0, 374.0, 372.0, 312.0, 282.0, 281.0, 280.0, 279.0, 266.0, 265.0, 264.0, 261.0, 219.0, 218.0, 215.0, 208.0, 205.0, 202.0, 189.0, 154.0, 152.0, 151.0, 150.0, 149.0, 146.0, 138.0, 117.0, 116.0, 114.0, 111.0, 110.0, 106.0, 105.0, 101.0, 100.0]


In [306]:
# Make list of all indices matching the specified frequencies

quotedRange = []

for f in topFreqs:
    quotedRange.append(np.where(tally == f)[0].tolist())

In [307]:
# Split sublists at non-consecutive indices (i.e. multiple quoted passages coincidentally with the same frequency)

res = []
tmp = []
prv = quotedRange[0][0]
for r in quotedRange:
    for l in r:
        if l-prv > 1:
            res.append(tmp)
            tmp = []
        tmp.append(l)
        prv = l
    res.append(tmp)

In [308]:
# Print frequently quoted passages with some context left and right

for n in range(len(res)):
    print(f"""
    {rawText[res[n][0]-100:res[n][0]]}
    \033[1m{rawText[res[n][0]:res[n][-1]]}\033[0m
    {rawText[res[n][-1]:res[n][-1]+100]}
    ---""")



    students of narrative.
JONATHAN CULLER
Ithaca, New York
5 See Derrida's Of Grammatology (Baltimore: 
    
     IN METHOD
Translated by Jane E. Lewin
Foreword by Jonathan Culler
CORNELL UNIVERSITY PRESS
ITHACA, 
    ---

    students of narrative.
JONATHAN CULLER
Ithaca, New York
5 See Derrida's Of Grammatology (Baltimore: 
    
     IN METHOD
Translated by Jane E. Lewin
Foreword by Jonathan Culler
CORNELL UNIVERSITY PRESS
ITHACA, 
    ---

    students of narrative.
JONATHAN CULLER
Ithaca, New York
5 See Derrida's Of Grammatology (Baltimore: 
    
     IN METHOD
Translated by Jane E. Lewin
Foreword by Jonathan Culler
CORNELL UNIVERSITY PRESS
ITHACA, 
    ---

    students of narrative.
JONATHAN CULLER
Ithaca, New York
5 See Derrida's Of Grammatology (Baltimore: 
    
     IN METHOD
Translated by Jane E. Lewin
Foreword by Jonathan Culler
CORNELL UNIVERSITY PRESS
ITHACA, 
    ---

    students of narrative.
JONATHAN CULLER
Ithaca, New York
5 See Derrida's Of Grammatology (Baltimore:

In [289]:
# ACTION: specify a phrase to drop here

dropPhrase = "Mrs. Dalloway said she would buy the flowers herself"

In [290]:
# Check location(s) of phrase

import re

phraseIndices = []

for match in re.finditer(dropPhrase, rawText, re.IGNORECASE):
    startIndex = match.start()
    endIndex = match.end()
    indexTuple = (startIndex, endIndex)
    phraseIndices.append(indexTuple)
    print(f"""Matched phrase at {startIndex}:{endIndex}\n
    {rawText[startIndex-100:startIndex]}\033[1m{rawText[startIndex:endIndex]}\033[0m{rawText[endIndex:endIndex+100]}""")

Matched phrase at 0:52

    Mrs. Dalloway said she would buy the flowers herself.

For Lucy had her work cut out for her. The doors would be taken off their hinges; Rumpelmayer's m


In [291]:
print(phraseIndices)

[(0, 52)]


In [294]:
# NOT YET WORKING

# Find "Locations in A" that contain any tuple from phraseIndices

df2 = df.explode(['Locations in A', 'Locations in B'])

df2[df2["Locations in A"].isin(phraseIndices)]

,creator,datePublished,Year,Decade,docSubType,docType,id,identifier,isPartOf,issueNumber,...,volumeNumber,wordCount,numMatches,Locations in A,Locations in B,doi,keyphrase,abstract,placeOfPublication,subTitle


In [283]:
# Explode lists of matches to be new entry each

for row in df['Locations in A'][3]:
    for tuple in row:
        if tuple == [1831, 2251]:
            print("Match detected!")
        else:
            print("No matches detected!")

No matches detected!
No matches detected!
No matches detected!
No matches detected!
No matches detected!
No matches detected!
No matches detected!
No matches detected!


In [ ]:
# Detect character ranges in Locations in A (+-1); report number of hits

In [2]:
# Delete character ranges from Locs in A and B
# Append string to existing list; save as csv; coded to work repeatedly as new phrases added

In [ ]:
# Save results jsonl with _phrasesdropped; coded to work repeatedly